In [50]:
import os
from tqdm import tqdm 
import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [51]:
path = '../input/dogs-vs-cats/train/train'

In [52]:
train = []

try:
    for i in tqdm(os.listdir(path)):
        img_path = os.path.join(path,i)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (50,50))
        if 'cat' in i:train.append([np.array(img), np.eye(2)[0]])
        elif 'dog' in i:train.append([np.array(img), np.eye(2)[1]])
            
except Exception as e:pass
        
np.random.shuffle(train)
np.save('train.npy',train)

100%|██████████| 25000/25000 [01:06<00:00, 374.53it/s]
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [53]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)     ## (#ip_channels, #op_channels, kernel_size)
        self.pool1 = nn.MaxPool2d((2,2))      # ((50-5) + 1)/2 = 23
        self.conv2 = nn.Conv2d(32, 64, 5)     # (23-5) + 1 = 19
        self.pool2 = nn.MaxPool2d((2,2))      # 19//2 = 9
        self.conv3 = nn.Conv2d(64, 128, 5)    # (9-5) + 1 = 5
        self.pool3 = nn.MaxPool2d((2,2))      # 5//2 = 2
        
        self.fc1 = nn.Linear(2*2*128, 256)
        self.fc2 = nn.Linear(256, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 2*2*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)
        
model = NN()

In [54]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

In [55]:
X = torch.Tensor([i[0] for i in train]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in train])

In [65]:
X.shape

torch.Size([25000, 50, 50])

In [56]:
test_size = int(len(X)*0.1)
test_size

2500

In [81]:
X_train = X[:-test_size]
y_train = y[:-test_size]
X_test = X[-test_size:]
y_test = y[-test_size:]

In [82]:
len(y_train)

22500

In [83]:
batch_size = 100
num_epochs = 1

for epoch in range(num_epochs):
    for i in tqdm(range(0, len(X_train), batch_size)):
        batch_X = X_train[i:batch_size+i].view(-1,1,50,50)
        batch_y = y_train[i:batch_size+i]
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
print(loss)

100%|██████████| 225/225 [00:59<00:00,  3.81it/s]

tensor(0.2103, grad_fn=<MseLossBackward>)


In [84]:
torch.argmax(torch.Tensor([34,2,3]))
#returns index w/ max value

tensor(0)

In [85]:
X_train.shape

torch.Size([22500, 50, 50])

In [86]:
X_test.shape

torch.Size([2500, 50, 50])

In [87]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(X_test))):
        original_idx = torch.argmax(y_test[i])
        output = model(X_test[i].view(-1,1,50,50))[0]
        predicted_idx = torch.argmax(output)
        if original_idx == predicted_idx:correct += 1
        total += 1
        
print('Accuracy', round(correct/total, 3))

100%|██████████| 2500/2500 [00:05<00:00, 445.16it/s]

Accuracy 0.669
